Gerekli kütüphaneleri yükleyelim.

In [ ]:
import pandas as pd  
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
filterwarnings('ignore')

Veri çerçevemizi bulunduğumuz dizinden yükleyelim ve bir veri çerçevesi haline getirerek df değişkenine atayalım. 

In [ ]:
df=pd.read_csv("../input/tablet/tablet.csv")

Veri çerçevesinin ilk 5 gözlemini yazdıralım.

In [ ]:
df.head()

Veri çerçevesinin kaç öznitelik ve kaç gözlemden oluştuğunu görüntüleyelim.

In [ ]:
df.shape

**Veri çerçevesi 2000 gözlem ve 20 öznitelikten oluşmaktadır.**

Veri çerçevesinin bilgilerini görüntüleyelim.

In [ ]:
df.info()

**Veri çerçevesi bellekte 312.6+ KB yer kaplamaktadır.OnKameraMP özniteliğinin karşısında yazan 1995 değeri, bu özniteliğin değerinin 1995'i için girildiğini,diğer 5'inin eksik olduğunu ifade etmektedir.RAM özniteliğinin karşısında yazan 1988 değeri, bu özniteliğin değerinin 1988'i için girildiğini,diğer 12'sinin eksik olduğunu ifade etmektedir.**

Veri çerçevemizin ne kadar dengeli dağıldığını sorgulayalım.

In [ ]:
df["FiyatAraligi"].value_counts()

**Her fiyat aralığının 500 adet gözlem barındırdığını görüyoruz. Bundan dolayı dengeli bir dağılım olduğunu söyleyebiliriz.**


Veri çerçevesinin dengeli dağıldığını gördük, şimdi de bunu görsel olarak ifade etmek için sns.countplot() fonksiyonuna FiyatAraligi parametresini verelim.

In [ ]:
sns.countplot(x="FiyatAraligi",data = df);

Korelasyon matrisi çizdirelim.

In [ ]:
corr=df.corr()
corr

**Korelasyon katsayısı -1 ≤ r ≤1 arasındadır. Korelasyon katsayısının 0’a yaklaşması değişkenler arasında zayıf ilişkinin varlığını gösterir. Değişkenler birlikte artıyor veya azalıyorsa pozitif yönde, değişkenlerden biri artarken diğeri azalıyorsa ise negatif yönde bir ilişki vardır.
Değerlere bakıldığında en güçlü pozitif ilişki OnKameraMP ve ArkaKameraMP değişkenleri arasındadır.**

Veri çerçevesindeki sayısal değişkenler için temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T 

**Standart sapma ile verilerin ne kadarının ortalamaya yakın olduğunu buluruz. Eğer standart sapma küçükse veriler ortalamaya yakın yerlerde dağılmışlardır. Bunun tersi olarak standart sapma büyükse veriler ortalamadan uzak yerlerde dağılmışlardır.Standart sapması en küçük olan değişkenimiz Kalinlik, verileri en düzgün dağılan değişkendir.Düzgün dağılımda Kalinlik'i sırasıyla,MikroislemciHizi,CekirdekSayisi,OnKameraMP,BataryaOmru,ArkaKameraMP,DahiliBellek,Agirlik,
CozunurlukGenislik,BataryaGucu,CozunurlukYükseklik ve son olarak RAM takip etmektedir.**

**Standart sapma, ortalamaya göre düşükse verilerin ayırt ediciliği düşüktür ve grup homojendir.Standart sapma, ortalamaya göre yüksekse grup heterojen ve verilerin ayırt ediciliği yüksektir.OnKameraMP heterojen diğer değişkenler ise homojendir.**


Kovaryans matrisi çizdirelim.

In [ ]:
df.cov()

**Kovaryans matrisi değişkenler arasındaki yönü belirler. Ancak ilişkinin şiddeti hakkında bir yorum yapamayız.**

Sayısal değişkenlerin dağılımını görmek için df üzerinden histogram çizdirelim.

In [ ]:
df.hist(figsize=(20,20));
plt.show()

Kategorik değişkenleri ideal tekniklerle ve araçlarla görselleştirelim. 

In [ ]:
plt.figure(figsize=(4,4))
colors=['pink','yellow']
df.Bluetooth.value_counts().plot.pie(autopct='%1.1f%%',colors=colors)
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
df.CiftHat.value_counts().plot.bar()
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
colors=['red','blue']
df.Dokunmatik.value_counts().plot.pie(autopct='%1.1f%%',colors=colors)
plt.show()

In [ ]:
plt.figure(figsize=(3,3))
sns.countplot(x="4G",data = df);
plt.show()

In [ ]:
plt.figure(figsize=(3,3))
sns.countplot(x="3G",data = df);
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
df.WiFi.value_counts().plot.pie(autopct='%1.1f%%')
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sns.countplot(x="Renk",data = df);
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
colors=['red','blue',"purple","green"]
df.FiyatAraligi.value_counts().plot.pie(autopct='%1.1f%%',colors=colors)
plt.show()

Veri çerçevesinde hangi öznitelikte kaç adet eksik değer olduğunu gözlemleyelim.

In [ ]:
df.isna().sum()

Değişkenlerdeki Null değerler yerine ortalama değerler yazdıralım.Daha sonra Null değerler kalmış mı kontrol edelim.

In [ ]:
eksik_OnKameraMP = df[df["OnKameraMP"].isnull()].index
eksik_OnKameraMP

In [ ]:
df.loc[eksik_OnKameraMP,"OnKameraMP"]= 4.312281 

In [ ]:
eksik_RAM = df[df["RAM"].isnull()].index
eksik_RAM

In [ ]:
df.loc[eksik_RAM,"RAM"]= 2116.243964

In [ ]:
df.isna().sum()

Değişkenlerde bulunan "Var" ve "Yok" ifadeleri bilgisayarın anlayacağı formatta değildir. Dolayısıyla label encoding yapmak için çağırdığımız kütüphaneden bir değişken oluşturalım ve Var, Yok değerleri için 0, 1 dönüşümü yapalım.

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
df['Bluetooth'] = label_encoder.fit_transform(df['Bluetooth'])
df.head()  #yok=1,var=0

In [ ]:
df['CiftHat'] = label_encoder.fit_transform(df['CiftHat'])
df.head()  #yok=1,var=0

In [ ]:
df['4G'] = label_encoder.fit_transform(df['4G'])
df.head()  #yok=1,var=0

In [ ]:
df['3G'] = label_encoder.fit_transform(df['3G'])
df.head()  #yok=1,var=0

In [ ]:
df['Dokunmatik'] = label_encoder.fit_transform(df['Dokunmatik'])
df.head()  #yok=1,var=0

In [ ]:
df['WiFi'] = label_encoder.fit_transform(df['WiFi'])
df.head()  #yok=1,var=0

get_dummies yardımıyla  Renk'e dair kategorik öznitelik çıkarımda bulunalım. Çünkü bu değişken nominal. Ordinal değil.

In [ ]:
df['Renk'] = pd.Categorical(df['Renk'])
dfDummies = pd.get_dummies(df['Renk'], prefix = 'Renk')
dfDummies

In [ ]:
df = pd.concat([df, dfDummies], axis=1)
df.head()

Renk özniteliğini silip dummy olarak yaratılan Renk'lerden de birisini hariç tutarak veri çerçevemizi güncelleyelim.

In [ ]:
df.drop(["Renk","Renk_Mor"],axis=1,inplace=True)

In [ ]:
df.head()

Veri çerçevemizi bağımlı ve bağımsız değişkenler olmak üzere bölütleyelim.

In [ ]:
y=df["FiyatAraligi"] #bağımlı
X=df.drop("FiyatAraligi",axis=1)#bağımsız değişken

Bağımlı ve bağımsız değişkenleri görüntüleyelim.

In [ ]:
y

In [ ]:
X

Bağımlı ve bağımsız değişkenleri %85 oranında eğitim ve %15 oranında test olmak üzere “train_test_split” metodu kullanılarak ayıralım.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15,random_state=42)

4 parça değişkeni kontrol edelim.

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
y_test

In [ ]:
y_train

KNN modelini parametre optimizasyonu yaparak kuralım ve eğitelim.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,50)}

En yakın komşu sayısını 2 ile 50 arasında hepsini deneyecek.

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(13)
knn_tuned = knn.fit(X_train, y_train) #Modeli eğitmek için bağımlı bağımsız değişkenlerden oluşturulan eğitim verilerini verelim

X_test gözlemlerini tahmin ettirelim ve y_pred'e atayalım.

In [ ]:
y_pred = knn_tuned.predict(X_test)

Doğruluk skoruna bakalım.

In [ ]:
accuracy_score(y_test, y_pred)

Karmaşıklık matrisini görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Modelin k değerini nasıl seçtiğini görüntülemek için, 48 adet k değeri için doğruluk skorlarını görselleştirelim.

In [ ]:
score_list = []

for each in range(2,50):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    score_list.append(knn2.score(X_test, y_test))
    
plt.plot(range(2,50),score_list)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")
plt.show()


Sınıflandırma raporu çizdirelim.

In [ ]:
print(classification_report(y_test, y_pred))

**Kesinlik (Precision) Positive olarak tahminlediğimiz değerlerin gerçekten kaç adedinin Positive olduğunu göstermektedir.Kesinlik değerinin yüksek olması bizim için model seçiminde önemli bir kriterdir.**

**Duyarlılık (Recall) Positive olarak tahmin etmemiz gereken işlemlerin ne kadarını Positive olarak tahmin ettiğimizi gösteren bir metriktir.Mümkün olduğunca yüksek olması gereklidir.**

**F1 Score değeri bize Kesinlik (Precision) ve Duyarlılık (Recall) değerlerinin harmonik ortalamasını göstermektedir.**

**Accuracy bir modelin başarısını ölçmek için çok kullanılan ancak tek başına yeterli olmadığı görülen bir metriktir.
Accuracy değeri modelde doğru tahmin ettiğimiz alanların toplam veri kümesine oranı ile hesaplanmaktır.**


GaussianNB modeli yaratalım ve eğitimi gerçekleştirelim.

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)

Modelin alabileceği parametreleri görüntüleyelim.

In [ ]:
nb_model

Model üzerinde yazılabilecek tüm komutları görüntüleyelim.

In [ ]:
dir(nb_model)

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

Doğruluk skorunu görüntüleyelim.

In [ ]:
accuracy_score(y_test, y_pred)

Karmaşıklık matrisini görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Sınıflandırma raporu çizdirelim.

In [ ]:
print(classification_report(y_test, y_pred))

Kesinlik skorunu görüntüleyelim.

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

Yakalama (Recall) veya Hassaslık (Sensitivity) skorunu görüntüleyelim.

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

F1 skorunu görüntüleyelim.

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

DecisionTree modelimizi inşa edelim ve eğitelim.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, y_train)

Modelin parametrelerini ve aldığı varsayılan değerleri görüntülemek için model ismini yazabiliriz.

In [ ]:
cart_model

Parametreleri optimize etmeden kurduğumuz bu model için tahminlerde bulunalım.

In [ ]:
y_pred = cart_model.predict(X_test)

Doğruluk skorunu görüntüleyelim.

In [ ]:
accuracy_score(y_test, y_pred)

Karmaşıklık matrisini görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Hedef özniteliğin değerlerine göre sınıflandırma raporunu görüntüleyelim.

In [ ]:
print(classification_report(y_test, y_pred))

**Accuracy-score başarı ölçütüne göre model başarı sıralaması; 0.92 ile knn modeli > 0.88 ile desicionTree modeli > 0.76 ile gaussianNB modeli şeklindedir.**